## ALS

Alernative Least Squares

https://medium.com/analytics-vidhya/model-based-recommendation-system-with-matrix-factorization-als-model-and-the-math-behind-fdce8b2ffe6d

In [ ]:
# Install the required packages
# be sure you have Java installed on your machine, https://www.java.com/en/download/
! pip install pyspark  
! pip install findspark

! pip install pandas

! pip install numpy
! pip install pymongo
! pip install python-dotenv

In [1]:
# Import the required packages
import pandas as pd
import numpy as np

import findspark
findspark.init()

import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

import os
java_home = os.environ.get('JAVA_HOME')
print(java_home)

from dotenv import dotenv_values
import pymongo

C:\Program Files\Java\jdk-11


In [3]:
# clear out the predicted ratings collection

# specify the name of the .env file name 
env_name = "myconfig.env" # following example.env template change to your own .env file name
config = dotenv_values(env_name)

# Azure Cosmos DB for MongoDB
cosmos_conn = config['cosmos_connection_string']
cosmos_client = pymongo.MongoClient(cosmos_conn)

db = cosmos_client["ProductRecommendation"]

# specify the collection to put the predicted ratings in
collection_predicted_rating = db['PredictedRating']

# delete all documents in the collection to clear it out
collection_predicted_rating.delete_many({})

DeleteResult({'ok': 1, 'n': 0}, acknowledged=True)

The data we start with here is a set of ratings that have been generated by users for products they have previously purchased.

In this cell we read the dataset of user-item ratings from the file, Augmented Ratings, into a Pandas DataFrame, selecting only relevant columns (UserId, ProductId, Rating). This DataFrame is then converted into a Spark DataFrame, which is required to run the ALS model.

In [2]:
# Create a Spark session
conf = SparkConf()
conf.set("spark.executor.memory","6g")
conf.set("spark.driver.memory", "6g")
conf.set("spark.driver.cores", "8")
sc = SparkContext.getOrCreate(conf)
spark = SparkSession.builder.getOrCreate()

# Load the data from the json file
full_pd_data = pd.read_json("./data/ratings/AugmentedRating.json")  # TO-DO change the name of this file
# just keep the required columns
pd_data = full_pd_data[['UserId', 'ProductId', 'Rating']]
# convert the data to spark dataframe, required to run the ALS model
data = spark.createDataFrame(pd_data)
# count the number of rows in the data
data.count()


200000

We next need to split the dataset into training and testing sets with an 80-20 split, ensuring that models are trained on a majority of the data while having a separate subset for evaluation. We then cache both to improve performance by storing them in memory.

In [4]:
# Split to create train (80%) and test (20%) datasets
train, test = data.randomSplit([0.8,0.2],10001)

#cache the train and test datasets
train.cache()
test.cache()

DataFrame[UserId: bigint, ProductId: bigint, Rating: double]

We now need to setup the ALS model, a collaborative filtering technique that will use the existing product ratings given by users over their purchased products to predict for each user the rating they might give to those products.

Hyperparameter tuning is performed through a grid search over a defined parameter space combined with cross-validation to ensure the model's generalizability. The CrossValidator in PySpark automates this process, evaluating the model's performance using RMSE (Root Mean Square Error) metric. RMSE measures the average difference bewteen predicted and actual values in a dataset.

After training, the best model is selected based on its performance, and its hyperparameters are printed for inspection. The model is then used to make predictions on the test set, and the RMSE of these predictions is calculated to assess the model's accuracy.

In [5]:
# we use the cross validator to tune the hyperparameters
als = ALS(
         userCol="UserId", 
         itemCol="ProductId",
         ratingCol="Rating", 
         coldStartStrategy="drop"
)

param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 100]) \
            .addGrid(als.regParam, [.1]) \
            .addGrid(als.maxIter, [10]) \
            .build()

evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="Rating", 
           predictionCol="prediction")

cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3, parallelism = 6)

In [6]:
# train the model itself
model = cv.fit(train)

# return the best model from those that were trained above
best_model = model.bestModel

In [9]:
# now take the train data and calculate how well it does predicting the ratings.
prediction = best_model.transform(test)
rmse = evaluator.evaluate(prediction)
print(f'RMSE = {rmse}. This is the average difference between the actual and predicted ratings. Lower values are better.')

RMSE = 0.3529248632229925. This is the average difference between the actual and predicted ratings. Lower values are better.


Gather the vectors for each user and product.

In [ ]:
# vector to describe what user attributes are important in making a predication for a rating
user_v = best_model.userFactors.collect()[0].features

# vector to describe what product attributes are important in making a prediction for a rating
item_v = best_model.itemFactors.collect()[0].features

print('User Vector: ' + str(user_v))
print('Product Vector: ' + str(item_v))

 The **dot product** of a user and item vector is computed as an example, showcasing how to predict a user's rating for a specific item based on their latent features.

In [12]:
np.dot((user_v),(item_v))

5.054752382127225

Instead of calculating dot product each time we need a prediction, to make our ecommerce app as fast as possible, we are going to first generate all of the recommendations for all users and all products. Then save these in Azure Cosmos DB for MongoDB

In [ ]:
# This line uses the best model obtained from the tuning process to generate recommendations for all users.
val_recommendations = best_model.recommendForAllUsers(10001)
rec_sys_final_predictions = []

for user in val_recommendations.collect():
    to_insert = {}
    to_insert['UserId'] = user.UserId
    to_insert['Predictions'] = [{"ProductId": user.recommendations[x].ProductId, "rating": user.recommendations[x].rating} for x in range(len(user.recommendations))]
    rec_sys_final_predictions.append(to_insert)

#collection_predicted_rating.insert_many(rec_sys_final_predictions)

In [15]:
val_recommendations.show()

+------+--------------------+
|UserId|     recommendations|
+------+--------------------+
|     1|[{42, 9.025747}, ...|
|    12|[{42, 8.798659}, ...|
|    22|[{42, 8.528533}, ...|
|    26|[{42, 7.185434}, ...|
|    27|[{42, 7.648633}, ...|
|    28|[{42, 8.761372}, ...|
|    31|[{42, 8.889992}, ...|
|    34|[{72, 7.499691}, ...|
|    44|[{72, 7.245907}, ...|
|    47|[{72, 7.309139}, ...|
|    52|[{42, 7.16784}, {...|
|    53|[{42, 9.005878}, ...|
|    65|[{72, 7.3831053},...|
|    76|[{42, 8.893301}, ...|
|    78|[{72, 7.526666}, ...|
|    81|[{42, 9.012387}, ...|
|    85|[{42, 7.038196}, ...|
|    91|[{42, 7.403313}, ...|
|    93|[{42, 8.873975}, ...|
|   101|[{42, 7.1201096},...|
+------+--------------------+
only showing top 20 rows

